# Actividad Autónoma 3 – Programación 2
## Implementación OOP de K-Nearest Neighbors (KNN) con el dataset *penguins*
## Pablo Andrés Terán Corrales


**Requisitos principales:**
- Encapsulamiento mediante atributos privados y getters/setters.
- Clase abstracta `KNNBase` con la firma de los métodos esenciales (`fit`, `distance`, `predict`).
- Subclase concreta `KNNClassifier` con la lógica específica del algoritmo KNN.
- Sobrecarga de operadores:
  - `__eq__`: dos ejemplares son iguales si tienen exactamente las mismas medidas.
  - `__add__`: combinación de dos listas de ejemplares en un único conjunto de entrenamiento.
- Implementación mínima de los métodos `fit`, `distance` (distancia euclidiana) y `predict`.
- Representación legible de un ejemplar con `__repr__`.
- Pruebas del clasificador con al menos tres valores de *k* (por ejemplo: 1, 3 y 5).

## Diseño orientado a objetos (UML textual simplificado)

**Clase abstracta `KNNBase`**
- Atributos protegidos/privados:
  - `_X_train: np.ndarray`
  - `_y_train: np.ndarray`
- Métodos abstractos:
  - `fit(X: np.ndarray, y: np.ndarray) -> None`
  - `distance(p1: np.ndarray, p2: np.ndarray) -> float`
  - `predict(X_new: np.ndarray, k: int = 3) -> np.ndarray`

**Clase `KNNClassifier(KNNBase)`**
- Implementa todos los métodos abstractos de `KNNBase`.
- Incluye validaciones básicas (dimensiones, tipos) y uso de NumPy.

**Clase `PenguinSample`**
- Atributos privados:
  - `__features: np.ndarray`  (medidas numéricas del pingüino)
  - `__species: str`          (etiqueta de especie)
- Getters/Setters controlados.
- Sobrecarga de operadores:
  - `__eq__(other)`  → igualdad si todas las medidas numéricas son idénticas.
  - `__repr__()`     → representación legible del ejemplar.

**Clase `PenguinDataset`**
- Atributos privados:
  - `__samples: list[PenguinSample]`
- Métodos:
  - `to_numpy()` → devuelve `(X, y)` listos para usar en `KNNClassifier`.
  - Sobrecarga `__add__(other)` para combinar dos datasets en uno nuevo.

In [1]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from abc import ABC, abstractmethod

try:
    import seaborn as sns
    SEABORN_AVAILABLE = True
except ImportError:
    SEABORN_AVAILABLE = False

SEABORN_AVAILABLE

True

In [3]:
class PenguinSample:
    """Representa un pingüino con medidas numéricas y su especie.

    Encapsula los atributos usando nombres privados y provee getters/setters.
    Dos ejemplares se consideran iguales si tienen exactamente las mismas
    medidas numéricas (features).
    """

    def __init__(self, features, species: str):
        features = np.asarray(features, dtype=float)
        self.__features = features
        self.__species = str(species)

    # Getters y setters controlados
    @property
    def features(self) -> np.ndarray:
        return self.__features

    @features.setter
    def features(self, value):
        value = np.asarray(value, dtype=float)
        if value.ndim != 1:
            raise ValueError("Las características deben ser un vector 1D")
        self.__features = value

    @property
    def species(self) -> str:
        return self.__species

    @species.setter
    def species(self, value: str):
        self.__species = str(value)

    def __eq__(self, other) -> bool:
        if not isinstance(other, PenguinSample):
            return NotImplemented
        return np.array_equal(self.__features, other.__features)

    def __repr__(self) -> str:
        feats_str = ", ".join(f"{v:.2f}" for v in self.__features)
        return f"PenguinSample(species='{self.__species}', features=[{feats_str}])"


class PenguinDataset:
    """Colección de PenguinSample con sobrecarga de __add__.

    __add__ permite combinar dos datasets en uno nuevo (unión de listas
    de ejemplares).
    """

    def __init__(self, samples=None):
        self.__samples = list(samples) if samples is not None else []

    @property
    def samples(self):
        return list(self.__samples)

    def add_sample(self, sample: PenguinSample):
        if not isinstance(sample, PenguinSample):
            raise TypeError("sample debe ser una instancia de PenguinSample")
        self.__samples.append(sample)

    def to_numpy(self):
        if not self.__samples:
            raise ValueError("El dataset está vacío")
        X = np.vstack([s.features for s in self.__samples])
        y = np.array([s.species for s in self.__samples], dtype=object)
        return X, y

    def __len__(self):
        return len(self.__samples)

    def __add__(self, other):
        if not isinstance(other, PenguinDataset):
            return NotImplemented
        return PenguinDataset(self.__samples + other.samples)

    def __repr__(self) -> str:
        return f"PenguinDataset(n_samples={len(self.__samples)})"

In [4]:
class KNNBase(ABC):
    """Clase base abstracta para clasificadores KNN."""

    def __init__(self):
        self._X_train: np.ndarray | None = None
        self._y_train: np.ndarray | None = None

    @abstractmethod
    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        pass

    @abstractmethod
    def distance(self, p1: np.ndarray, p2: np.ndarray) -> float:
        pass

    @abstractmethod
    def predict(self, X_new: np.ndarray, k: int = 3) -> np.ndarray:
        pass


class KNNClassifier(KNNBase):
    """Clasificador KNN usando distancia euclidiana y NumPy."""

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)
        if X.ndim != 2:
            raise ValueError("X debe ser un arreglo 2D (n_muestras, n_características)")
        if len(X) != len(y):
            raise ValueError("X e y deben tener el mismo número de filas")
        self._X_train = X
        self._y_train = y

    def distance(self, p1: np.ndarray, p2: np.ndarray) -> float:
        p1 = np.asarray(p1, dtype=float)
        p2 = np.asarray(p2, dtype=float)
        if p1.shape != p2.shape:
            raise ValueError("Los vectores deben tener la misma dimensión para calcular la distancia")
        return float(np.linalg.norm(p1 - p2))

    def _check_is_fitted(self):
        if self._X_train is None or self._y_train is None:
            raise RuntimeError("El clasificador aún no ha sido entrenado. Llama a fit() primero.")

    def predict(self, X_new: np.ndarray, k: int = 3) -> np.ndarray:
        self._check_is_fitted()
        X_new = np.asarray(X_new, dtype=float)
        if X_new.ndim == 1:
            X_new = X_new.reshape(1, -1)
        if k <= 0:
            raise ValueError("k debe ser un entero positivo")
        if k > len(self._X_train):
            raise ValueError("k no puede ser mayor que el número de muestras de entrenamiento")

        predictions = []
        for x in X_new:
            # Distancias a todos los puntos de entrenamiento
            diffs = self._X_train - x
            dists = np.linalg.norm(diffs, axis=1)
            # Índices de los k vecinos más cercanos
            nn_indices = np.argsort(dists)[:k]
            nn_labels = self._y_train[nn_indices]
            # Votación mayoritaria
            values, counts = np.unique(nn_labels, return_counts=True)
            majority_label = values[np.argmax(counts)]
            predictions.append(majority_label)
        return np.array(predictions, dtype=object)


## Carga y preparación del dataset `penguins`

En esta sección se carga el dataset de pingüinos. Se usan cuatro variables numéricas habituales:

- `bill_length_mm`
- `bill_depth_mm`
- `flipper_length_mm`
- `body_mass_g`

Se eliminan filas con valores faltantes y se construye un `PenguinDataset`.

In [5]:
if SEABORN_AVAILABLE:
    penguins = sns.load_dataset('penguins')
else:
    raise ImportError(
        "Seaborn no está disponible en este entorno. "
        "En tu computador puedes instalarlo con 'pip install seaborn' "
        "o cargar manualmente un CSV con el dataset penguins."
    )

penguins = penguins.dropna(subset=['bill_length_mm', 'bill_depth_mm',
                                   'flipper_length_mm', 'body_mass_g',
                                   'species'])
features_cols = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

samples = []
for _, row in penguins.iterrows():
    feats = row[features_cols].values.astype(float)
    species = row['species']
    samples.append(PenguinSample(feats, species))

dataset_full = PenguinDataset(samples)
dataset_full, len(dataset_full)

(PenguinDataset(n_samples=342), 342)

### Demostración de `__eq__` y `__add__`

- `__eq__`: compara dos `PenguinSample` por sus medidas numéricas.
- `__add__`: combina dos `PenguinDataset` en uno nuevo.

In [6]:
# Tomamos algunos ejemplares para probar las sobrecargas
sample_a = dataset_full.samples[0]
sample_b = PenguinSample(sample_a.features.copy(), sample_a.species)
sample_c = dataset_full.samples[1]

print("sample_a:", sample_a)
print("sample_b (copia de a):", sample_b)
print("sample_c (distinto):", sample_c)
print("sample_a == sample_b?", sample_a == sample_b)
print("sample_a == sample_c?", sample_a == sample_c)

# Demostración de __add__ en PenguinDataset
dataset_1 = PenguinDataset(dataset_full.samples[:5])
dataset_2 = PenguinDataset(dataset_full.samples[5:10])
dataset_combined = dataset_1 + dataset_2
dataset_1, dataset_2, dataset_combined

sample_a: PenguinSample(species='Adelie', features=[39.10, 18.70, 181.00, 3750.00])
sample_b (copia de a): PenguinSample(species='Adelie', features=[39.10, 18.70, 181.00, 3750.00])
sample_c (distinto): PenguinSample(species='Adelie', features=[39.50, 17.40, 186.00, 3800.00])
sample_a == sample_b? True
sample_a == sample_c? False


(PenguinDataset(n_samples=5),
 PenguinDataset(n_samples=5),
 PenguinDataset(n_samples=10))

## Entrenamiento del KNNClassifier y pruebas con distintos valores de *k*

A continuación se divide el dataset en entrenamiento y prueba, se entrena el clasificador KNN y se generan predicciones para *k* = 1, 3 y 5.

In [7]:
# Conversión a NumPy para usar en KNNClassifier
X, y = dataset_full.to_numpy()
n_samples = len(X)

# División simple entrenamiento/prueba (80% / 20%)
rng = np.random.default_rng(42)
indices = rng.permutation(n_samples)
train_size = int(0.8 * n_samples)
train_idx, test_idx = indices[:train_size], indices[train_size:]

X_train, y_train = X[train_idx], y[train_idx]
X_test, y_test = X[test_idx], y[test_idx]

classifier = KNNClassifier()
classifier.fit(X_train, y_train)
print("Tamaño entrenamiento:", X_train.shape, "Tamaño prueba:", X_test.shape)

Tamaño entrenamiento: (273, 4) Tamaño prueba: (69, 4)


In [8]:
# Predicciones para k = 1, 3 y 5
ks = [1, 3, 5]
results = {}
for k in ks:
    y_pred = classifier.predict(X_test, k=k)
    results[k] = y_pred

# Tabla con algunas predicciones
df_results = pd.DataFrame({
    'true_species': y_test
})
for k in ks:
    df_results[f'pred_k_{k}'] = results[k]

df_results.head(15)

,true_species,pred_k_1,pred_k_3,pred_k_5
0,Adelie,Adelie,Adelie,Adelie
1,Gentoo,Gentoo,Gentoo,Gentoo
2,Adelie,Adelie,Adelie,Adelie
3,Chinstrap,Chinstrap,Chinstrap,Chinstrap
4,Gentoo,Chinstrap,Gentoo,Gentoo
5,Gentoo,Gentoo,Gentoo,Gentoo
6,Adelie,Adelie,Adelie,Adelie
7,Adelie,Adelie,Adelie,Adelie
8,Adelie,Adelie,Adelie,Adelie
9,Adelie,Chinstrap,Chinstrap,Chinstrap


### Comentario sobre el efecto de *k*

- Con **k = 1**, el clasificador es muy sensible al ruido: cada punto de prueba toma la clase de su vecino más cercano, lo que puede producir más variabilidad.
- Con **k = 3** y **k = 5**, la decisión se suaviza al considerar más vecinos. Esto suele hacerlo más robusto frente a outliers, pero si *k* es demasiado grande, puede perder capacidad para capturar fronteras de decisión más complejas.

En el dataframe de resultados se pueden comparar las columnas `pred_k_1`, `pred_k_3` y `pred_k_5` para ver cómo cambian las predicciones para los mismos ejemplares de prueba.